In [155]:
import sqlite3
import pandas as pd
import folium
from folium.plugins import MarkerCluster
conn = sqlite3.connect(r"..\databases\ubereats.db")
cursor = conn.cursor()
conn.close()


In [131]:
query =  '''
        SELECT distinct name ,latitude,longitude
        FROM restaurants   
        where postal_code != ''
        ;    
'''

# for row in cursor.execute(query):
#         print(row)
cursor.execute(query)
data = cursor.fetchall()

#df = pd.DataFrame(data, columns=['name', 'latitude', 'longitude','postal_code'])
df = pd.read_sql_query(query, conn)
df.head(10)

# JOIN locations_to_restaurants ON locations_to_restaurants.restaurant_id = restaurants.id
# JOIN locations ON locations.id = locations_to_restaurants.location_id

,name,latitude,longitude
0,Meli Mêly,50.855476,4.399550
1,'t Misverstand,50.794531,4.334645
2,Mumbaï Dreams,50.792346,4.352460
3,Gri Gri,50.792341,4.351862
4,Le Nénuphar,50.848879,4.433540
5,Guignol,50.793018,4.347961
6,Thai Smile,50.794881,4.364859
7,Le Touareg Uccle,50.794579,4.364152
8,La Soeur du Patron Auderghem,50.816075,4.430371
9,Sushi Mart,50.845422,4.436319


In [ ]:
query_deliveroo = """
SELECT
    locations.id, locations.name, locations.longitude, locations.latitude,
    COUNT(locations_to_restaurants.location_id) AS count_per_location
FROM
    locations
JOIN locations_to_restaurants ON locations.id = locations_to_restaurants.location_id

GROUP BY
    locations.id
ORDER BY
	locations.name;
"""

query_takeaway = """ 
SELECT
    locations.id, locations.name, locations.longitude, locations.latitude,
    COUNT(locations_to_restaurants.location_id) AS count_per_location
FROM
    locations
JOIN locations_to_restaurants ON locations.id = locations_to_restaurants.location_id

GROUP BY
    locations.id
ORDER BY
	locations.name;
"""
query_ubereats = """ 
SELECT
    locations.id, locations.name, locations.longitude, locations.latitude,
    COUNT(locations_to_restaurants.location_id) AS count_per_location
FROM
    locations
JOIN locations_to_restaurants ON locations.id = locations_to_restaurants.location_id

GROUP BY
    locations.id
ORDER BY
	locations.name
"""
cursor.execute(query_ubereats)
data = cursor.fetchall()

In [133]:
import geopandas as gpd
import pandas as pd

import matplotlib.pyplot as plt
from shapely.geometry import Point


In [ ]:
test = gpd.read_file(r"..\assets\geo\gadm41_BEL_1.shp")

crs = {'init':'EPSG:4326'}
geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
geo_df = gpd.GeoDataFrame(df, 
                          crs = crs, 
                          geometry = geometry)
geo_df.head()
fig, ax = plt.subplots(figsize = (10,10))
test.to_crs(epsg=4326).plot(ax=ax, color='grey')
geo_df.plot(ax=ax)
ax.set_title('Restaurants in Belgium')

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point

# Read in the shapefile for the administrative boundaries (Belgium)
test = gpd.read_file(r"..\assets\geo\gadm41_BEL_3.shp")

#  Prepare GeoDataFrame for restaurants
# Create the geometry for restaurants (Point objects)
geometry = [Point(lon, lat) for lon, lat in zip(df['longitude'], df['latitude'])]

# Set the CRS (Coordinate Reference System) to EPSG:4326
crs = {'init': 'epsg:4326'}

# Create a GeoDataFrame for restaurants
geo_df = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)

# Plot the map with boundaries and restaurant points
# Create a figure and axis for the plot
fig, ax = plt.subplots(figsize=(12, 12))  # Larger plot for better visualization

# Plot the Belgium administrative boundaries (Level 1)
test.to_crs(epsg=4326).plot(ax=ax, color='lightgrey', edgecolor='black', linewidth=0.5)

# Plot the restaurant data on top of the boundaries
geo_df.plot(ax=ax, color='blue', markersize=10, alpha=0.6, label="Restaurants")

#  Customize the map appearance
ax.set_title("Restaurant Distribution in Belgium", fontsize=16, fontweight='bold')
ax.set_xlabel("Longitude", fontsize=12)
ax.set_ylabel("Latitude", fontsize=12)

# Add a legend
ax.legend(loc="upper right", fontsize=10)

# Adjust the x and y limits for better zoom and layout
ax.set_xlim([df['longitude'].min() - 0.5, df['longitude'].max() + 0.5])
ax.set_ylim([df['latitude'].min() - 0.5, df['latitude'].max() + 0.5])


plt.tight_layout() 
plt.show()


In [136]:
import folium
import geopandas as gpd
from shapely.geometry import Point
import webbrowser

#Read shapefile for Belgium boundaries
test = gpd.read_file(r"..\assets\geo\gadm41_BEL_2.shp")

# Prepare GeoDataFrame for restaurant locations
geometry = [Point(lon, lat) for lon, lat in zip(df['longitude'], df['latitude'])]
crs = {'init': 'epsg:4326'}
geo_df = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)

# Initialize the Folium map (centered around Belgium)
m = folium.Map(location=[50.8503, 4.3517], zoom_start=7)  # Centered on Belgium with zoom level 7

#Add Belgium boundary (GeoJSON) to the map
# Convert the GeoDataFrame to GeoJSON and add it as a layer
folium.GeoJson(test.to_crs(epsg=4326)).add_to(m)

#Add restaurant locations as markers on the map
for _, row in geo_df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Restaurant: {row['name']}",  
        icon=folium.Icon(color='blue', icon='cutlery')
    ).add_to(m)

# for _, row in df.iterrows():
#     folium.CircleMarker(
#         location=[row['latitude'], row['longitude']],
#         radius=5,  # Size of the dot
#         color='blue',  # Color of the border
#         fill=True,
#         fill_color='blue',  # Color of the filled circle
#         fill_opacity=0.6,  # Transparency of the filled circle
#         popup=f"Restaurant: {row['name']}"
#     ).add_to(m)


m.save('restaurants_map.html')  # Save the map to an HTML file

map_path = r"..\assets\geo\restaurants.html"
m.save(map_path)

# Open map in browser
webbrowser.open(map_path)


c:\Users\Vinay\Desktop\Becode\Projects\delivery-market-analysis-with-SQL\.venv\Lib\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


True

In [154]:
import folium
import pandas as pd
import branca

df = pd.read_sql_query(query_ubereats,conn)

# Step : Define count ranges and assign colors
def price_color(count):
    if count < 30:
        return 'green'
    elif count < 100:
        return 'orange'
    elif count < 300:
        return 'red'
    else:
        return 'darkred'

# Step : Create a map with Folium
m = folium.Map(location=[50.5039, 4.4699], zoom_start=8)  # Centered on Belgium

# Step : Adding markers for each point of interest with count per location information
for i, row in df.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,  # You can adjust the size of the marker based on the average price
        popup=f"PoI Name: {row['name']}<br>Restaurants: {row['count_per_location']}",
        color=price_color(row['count_per_location']),  # Use the function to determine the color
        fill=True,
        fill_color=price_color(row['count_per_location']),
        fill_opacity=0.6
    ).add_to(m)

# Step : Create a legend
color_scale = branca.colormap.StepColormap(['green', 'orange', 'red', 'darkred'], 
                                             vmin=0, vmax=821, 
                                             index=[0, 30, 100, 300, 821], 
                                             caption='Number of restaurants around PoI')

# Add the legend to the map
color_scale.add_to(m)

# Step : Save the map to an HTML file
m.save("../assets/geo/count_per_location_map_ubereats.html")

# You can now open the HTML file to see the map




In [138]:
import folium
import geopandas as gpd
from shapely.geometry import Point
from folium.plugins import HeatMap  # Import the HeatMap plugin
import webbrowser

# Read shapefile for Belgium boundaries
test = gpd.read_file(r"..\assets\geo\gadm41_BEL_2.shp")

# Prepare GeoDataFrame for restaurant locations
geometry = [Point(lon, lat) for lon, lat in zip(df['longitude'], df['latitude'])]
crs = {'init': 'epsg:4326'}
geo_df = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)

# Initialize the Folium map (centered around Belgium)
m = folium.Map(location=[50.8503, 4.3517], zoom_start=7)  # Centered on Belgium with zoom level 7

# Add Belgium boundary (GeoJSON) to the map
# Convert the GeoDataFrame to GeoJSON and add it as a layer
folium.GeoJson(test.to_crs(epsg=4326)).add_to(m)

# Prepare data for HeatMap: List of [latitude, longitude]
heat_data = [[row['latitude'], row['longitude']] for _, row in geo_df.iterrows()]

# Add HeatMap layer to the map
HeatMap(heat_data, radius=15, blur=10).add_to(m)

# Save the map with heatmap to an HTML file
map_path = r"..\assets\geo\restaurants_heatmap_only.html"
m.save(map_path)

# Open map in browser
webbrowser.open(map_path)


c:\Users\Vinay\Desktop\Becode\Projects\delivery-market-analysis-with-SQL\.venv\Lib\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


True